<a href="https://colab.research.google.com/github/fractalsproject/spacenet6challenge/blob/master/notebooks/SpaceNet6Challenge_Baseline_EvalSAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#
# Config
#

# Force cloning a fresh repo
force_clone = True

# Top-level directory of pretest/pretain preproc artifacts
pretrain_path = '/content/mountOnColab/pretrain'

# Top-level directory of training sessions
train_sessions_path = '/content/mountOnColab/train_sessions'

# Where to get the yaml and csv templates
configs_path = './spacenet6challenge/configs'

# The (string) date of the training session
session_date = '2020-05-06_15_34_04.994493' 

# The name of the weights file to load
weights_file_name = 'best_epoch1_1.159000039100647.model'

# Set to True if adding/changing solaris code in this Colab session
reinstall_solaris = False



In [2]:
# Clone the spacenet6challenge project
if force_clone:
  print("Removing local 'spacenet6challenge' repo if it exists")
  !if [ -d "spacenet6challenge" ]; then rm -r "spacenet6challenge"; fi
!if [ ! -d "spacenet6challenge" ]; then git clone --recursive "https://github.com/fractalsproject/spacenet6challenge.git" ; else echo "spacenet6challenge directory already exists"; fi

Removing local 'spacenet6challenge' repo if it exists
Cloning into 'spacenet6challenge'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 507 (delta 113), reused 113 (delta 46), pack-reused 319
Receiving objects: 100% (507/507), 134.29 KiB | 2.69 MiB/s, done.
Resolving deltas: 100% (245/245), done.
Submodule 'CosmiQ_SN6_Baseline' (https://github.com/CosmiQ/CosmiQ_SN6_Baseline/) registered for path 'CosmiQ_SN6_Baseline'
Submodule 'solaris' (https://github.com/fractalsproject/solaris.git) registered for path 'solaris'
Cloning into '/content/spacenet6challenge/CosmiQ_SN6_Baseline'...
remote: Enumerating objects: 323, done.        
remote: Total 323 (delta 0), reused 0 (delta 0), pack-reused 323        
Receiving objects: 100% (323/323), 81.24 MiB | 35.10 MiB/s, done.
Resolving deltas: 100% (187/187), done.
Cloning into '/content/spacenet6challenge/solaris'...
remote: Enumerating objec

In [3]:
# Check if we need to do a full package installation
try:
  import solaris
  from solaris import utils
  print("Found solaris package.  Assuming a previous installation worked.")
  solaris_ok = True
except:
  import sys
  sys.path.append('/content/spacenet6challenge')
  import spacenet6.colab.setup
  spacenet6.colab.setup.baseline_prereqs(force=True)
  solaris_ok = False

Checking for jupyter/ipython environment...
Passed.
Checking for Colab environment...
Passed.
CMDS ['sudo apt-get install -y software-properties-common ', 'sudo add-apt-repository -y ppa:ubuntugis/ppa && apt-get -y update', 'sudo apt-get install -y libspatialindex-dev python-rtree gdal-bin', 'sudo apt-get install -y build-essential libssl-dev libffi-dev libxml2-dev libxslt1-dev zlib1g-dev', 'sudo apt-get install -y libgdal-dev ', 'sudo gdal-config --version ', 'pip install affine>=2.3.0 ', 'pip install albumentations==0.4.3', 'pip install fiona>=1.7.13 ', 'pip install geopandas>=0.7.0', 'pip install matplotlib>=3.1.2', 'pip install networkx>=2.4 ', 'pip install opencv-python>=4.1', 'pip install pandas>=0.25.3 ', 'pip install pyproj>=2.1 ', 'pip install pyyaml==5.2', 'pip install rasterio>=1.0.23', 'pip install requests==2.22.0', 'pip install rtree>=0.9.3 ', 'pip install scikit-image>=0.16.2', 'pip install scipy>=1.3.2 ', 'pip install torchvision>=0.5.0', 'pip install tqdm>=4.40.0 ', 'p

OK.
Running command: "pip install rtree>=0.9.3 ".  Please wait...
OK.
Running command: "pip install scikit-image>=0.16.2".  Please wait...
OK.
Running command: "pip install scipy>=1.3.2 ".  Please wait...
OK.
Running command: "pip install torchvision>=0.5.0".  Please wait...
OK.
Running command: "pip install tqdm>=4.40.0 ".  Please wait...
OK.
Running command: "pip install urllib3>=1.25.7".  Please wait...
OK.
Running command: "pip install gdal>=3.0.2 ".  Please wait...
OK.
Running command: "pip install tensorflow-gpu==1.13.1".  Please wait...
OK.
CMDS ['cd /content/spacenet6challenge/solaris && pip install .'] False
Running command: "cd /content/spacenet6challenge/solaris && pip install .".  Please wait...

All baseline prereqs have been installed.


In [0]:
# Useful for print debugging solaris
if reinstall_solaris:
  !pip uninstall solaris
  !cd /content/spacenet6challenge/solaris && python setup.py clean
  !cd /content/spacenet6challenge/solaris && pip install .
  import solaris
  import importlib
  importlib.reload(solaris)
  solaris_ok = True

In [5]:
# TODO:  Why do we need to do this here...
if not solaris_ok:
  !pip install shapely
  !cd /content/spacenet6challenge/solaris && python setup.py clean
  !cd /content/spacenet6challenge/solaris && pip install .
  try:
    #from solaris import utils
    #print("Found solaris package.  Assuming installation worked.")
    solaris_ok = True
  except:
    raise Exception("You should restart the kernel and run the notebook again.")

  Cloning git://github.com/toblerity/shapely (to revision master) to /tmp/pip-req-build-a5r6h131
  Running command git clone -q git://github.com/toblerity/shapely /tmp/pip-req-build-a5r6h131
  Created wheel for Shapely: filename=Shapely-1.8.dev0-cp36-cp36m-linux_x86_64.whl size=664959 sha256=5641540278c4f4ee1ac6eae9b6b67352e51bc8e241852230a0fb1fe3c7a0e8ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-cgjncjbn/wheels/b5/64/12/74e189eed9f8e46360196a89a3c2572059194e4f48b2daca95
Successfully built Shapely
  Found existing installation: Shapely 1.7.0
    Uninstalling Shapely-1.7.0:
      Successfully uninstalled Shapely-1.7.0
running clean
Processing /content/spacenet6challenge/solaris
     |████████████████████████████████| 71kB 2.5MB/s 
     |████████████████████████████████| 133kB 9.7MB/s 
     |████████████████████████████████| 92.5MB 33kB/s 
  Created wheel for solaris: filename=solaris-0.2.2-cp36-none-any.whl size=14909551 sha256=082fda135808e56b44cb64cebee88a2a4c3f5e666697e10d849

In [6]:
import solaris
import importlib
importlib.reload(solaris)
import solaris
dir(solaris)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'bin',
 'data',
 'eval',
 'nets',
 'raster',
 'tile',
 'utils',
 'vector']

In [7]:
# Check tensorflow 
import tensorflow
print("tensorflow version=",tensorflow.version.VERSION)
if (tensorflow.version.VERSION!='1.13.1'):
  raise Exception("You need to restart the kernel and resume from here.")

# Check torch 
import torch
print( "torch version=", torch.version.__version__) 
if not torch.version.__version__.startswith("1.5"):
  raise Exception("You need restart the kernel and resume from here.")
if not torch.cuda.is_available():
  raise Exception("Torch cuda is not available.")
else:
  print("Torch cuda is available.")

# Check all installation packages are available
try:
  import solaris
  from solaris import utils
  import sys
  !pip install geopandas>=0.7.0
  import geopandas
  sys.path.append('/content/spacenet6challenge/CosmiQ_SN6_Baseline')
  import baseline
  sys.path.append('/content/spacenet6challenge')
  from spacenet6.cosmiq import baseline_wrap
except:
  raise Exception("Installation checks failed.")

print("Installation checks passed.")

tensorflow version= 1.13.1
torch version= 1.5.0+cu101
Torch cuda is available.
Installation checks passed.


In [8]:
# Mount GCP bucket data
import sys
sys.path.append('./spacenet6challenge')
import spacenet6.colab.bucket
spacenet6.colab.bucket.mount("spacenet_challenge_data",force_new_mount=True)

Checking for jupyter/ipython environment...
Passed.
Checking for colab environment...
Passed.
Trying to authenticate GCP user...
Running command: "echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list".  Please wait...
OK.
Running command: "curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - ".  Please wait...
OK.
Running command: "apt -qq update".  Please wait...
OK.
Running command: "apt -qq install gcsfuse".  Please wait...
OK.
Running command: "mkdir mountOnColab".  Please wait...
OK.
Running command: "gcsfuse --implicit-dirs spacenet_challenge_data mountOnColab".  Please wait...
OK.
Done. Getting folder contents...
Running command: "ls -als /content/mountOnColab".  Please wait...

Mount was successful.


True

In [0]:
#
# Do some config checks
#
import os

# pretrain path
if not os.path.exists(pretrain_path):
  raise Exception("pretrain path does not exist.")

# train sessions toplevel
if not os.path.exists(train_sessions_path):
  raise Exception("train sessions path does not exist.")

# configs dir for yaml templates and csv's
if not os.path.exists(configs_path):
  raise Exception("configs path does not exist.")

# Dated training sessions directory containing weights
if not os.path.exists( os.path.join( train_sessions_path, session_date, "weights" ) ):
  raise Exception("No session dir %s with weights" % session_date )

# Name of weights file
weights_path = os.path.join( train_sessions_path, session_date, "weights", weights_file_name )
if not os.path.exists( weights_path ):
  raise Exception("Invalid weights path-> %s" % weights_path )

In [12]:
# Eval on SAR model
import os
import datetime
import solaris as sol
import model

# create a dir for this test session
dpath = str(datetime.datetime.now()).replace(":","_").replace(" ","_")
outpath = '%s/%s/%s' % ( train_sessions_path, "test_sessions", dpath )
os.makedirs(outpath,exist_ok=True)

# create test.csv from template
f = open( os.path.join(configs_path,'test.csv') )
txt = f.read()
f.close()
newtxt = txt.replace('./root',pretrain_path)
traincsv = os.path.join( outpath, "test.csv")
f = open(testcsv, 'w')
f.write(newtxt)
f.flush()
f.close()

# create an eval yaml from template
f = open( os.path.join(configs_path, 'eval_sar.yaml') )
txt = f.read()
f.close()
newtxt = txt.replace("DATEDIR",outpath)
newtxt = newtxt.replace("TESTCSV",testcsv)
newyaml = os.path.join( outpath, "eval_sar.yaml")
f = open(newyaml,'w')
f.write(newtxt)
f.flush()
f.close()

print("Wrote file->%s" % newyaml)

# create a test config from yaml
config = sol.utils.config.parse(newyaml)
config['pretrained'] = False

sar_dict = {
    'model_name': 'unet11',
    'weight_path': None,
    'weight_url': None,
    'arch': model.UNet11
}

# test
#trainer = sol.nets.train.Trainer(config, custom_model_dict=sar_dict)
#trainer.train()

FileNotFoundError: ignored

In [0]:
# Evaluate on test data

if False:

  #model output directory= /content/mountOnColab/train_sessions/2020-05-03_14_04_02.011290
  #model weights directory= /content/mountOnColab/train_sessions/2020-05-03_14_04_02.011290/weights

  # Verify all the directories
  import os
  import datetime
  import importlib
  import sys
  sys.path.append('./spacenet6challenge/CosmiQ_SN6_Baseline')
  sys.path.append('./spacenet6challenge/')
  from spacenet6.cosmiq import baseline_wrap
  import baseline
  importlib.reload(baseline)
  import importlib
  importlib.reload(baseline_wrap)

  # where is the raw test SAR data?
  test_path = "/content/mountOnColab/SAR/test_public/AOI_11_Rotterdam"
  if not os.path.exists(test_path):
    raise Exception("Could not find training data")

  # where is the pretrain_pretest data?
  pretrain_pretest_path = '/content/mountOnColab/pretrain'
  if not os.path.exists(pretrain_pretest_path):
    raise Exception('Could not find pretrain data')

  # validate the directory where we load the weights.
  if not os.path.exists( os.path.join(read_model_weights_toplevel_dir, "weights" ) ):
    raise Exception("model weights dir does not exist.")

  # where to put output from this test eval session?
  test_sessions_path = '/content/mountOnColab/tests_sessions'
  dpath = str(datetime.datetime.now()).replace(":","_").replace(" ","_")
  outpath = '%s/%s' % ( test_sessions_path, dpath )
  os.makedirs( os.path.join( outpath, "sartest"), exist_ok=True )
  os.makedirs( os.path.join( outpath, "inference_continuous"), exist_ok=True )
  os.makedirs( os.path.join( outpath, "inference_binary"), exist_ok=True )
  os.makedirs( os.path.join( outpath, "inference_vectors"), exist_ok=True )

  cmdargs = [ "--test",
                  "--testdir", "%s/SAR-Intensity" % test_path,
                              "--rotationfilelocal","%s/SAR_orientations.txt" % pretrain_pretest_path,
                              "--maskdir", "%s/masks" % pretrain_pretest_path,
                              "--sarprocdir", "%s/sartrain" % pretrain_pretest_path, 
                              "--opticalprocdir","%s/optical" % pretrain_pretest_path,
                              "--testcsv","%s/test.csv" % pretrain_pretest_path,
                              "--yamlpath","%s/infer.yaml" % pretrain_pretest_path,
                              "--modeldir","%s/weights" % read_model_weights_toplevel_dir,
                              "--testprocdir","%s/sartest" % outpath,
                              "--testoutdir","%s/inference_continuous" % outpath,
                              "--testbinarydir","%s/inference_binary" % outpath,
                              "--testvectordir","%s/inference_vectors" % outpath,
                              "--rotate",
                              "--mintestsize","80"]
  args = baseline_wrap.parse_args(cmdargs)
  #print(args)
  #import importlib
  #importlib.reload(baseline_wrap)
  print(vars(args))
  baseline_wrap.invoke(outpath, "/content/mountOnColab/pretrain", args)